# Melvin data cleanup
## Situatie
- Ruwe data betreft 3 tabellen:
  - Situation: beschrijving van de wegwerkzaamheid
  - Restriction (mogelijk meerdere per sitation): combinatie van locaties, voertuigtypes, rijrichting, beperking (afgesloten, snelheid)
  - Periods (mogelijk meerdere per situation): datum/tijd
- Veel duplicates
- overtollige kolommen
## Doelen
- Restrictions filteren naar volledige afsluitingen
- Restrictions geometry naar getroffen wegvakken omzetten
- Gecombineerde tabel maken met situation (id, descriptions, etc.) + wegvak + period
- Gecombineerde tabel omzetten naar 'virtuele verkeersborden'
## Nog niet geimplementeerd
- Het omzetten van Melvin data naar 'virtuele verkeersborden' incl. alle beperkingen en mogelijkheden (nu is de aanname dat de weg altijd volledig dicht is)

In [3]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt",user="user",host="host",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

## Situation table

In [4]:
sql='''DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_situation_cleaned;
CREATE TABLE int_conflicten.ndw_werkzaamheden_situation_cleaned AS (
    select distinct id as melvin_id
    ,s."properties.workObject" as workObject
    ,s."properties.impact" as impact
    ,s."properties.delay" as expectedDelay
    ,s."properties.location.city" as city
    ,s."properties.location.road" as roadname
    ,s."properties.location.district" as district
    ,s."properties.activityType" as activityType
    ,s."properties.roadAuthority.name" as roadAuthority
    ,s."properties.roadAuthority.type" as roadAuthorityType
    ,s."properties.situationRecordId" as sit_rec_id
    ,longitude as lon
    ,latitude as lat
    from stg_conflicten.ndw_werkzaamheden_situation_raw as s
);'''
cur.execute(sql)

## Restriction table

In [5]:
sql='''DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_restriction_cleaned;
CREATE TABLE int_conflicten.ndw_werkzaamheden_restriction_cleaned AS (
    select r.*
    -- , r3.geom
    from (
        select distinct id as melvin_id
        ,s."properties.restrictionType" as restrictionType
        ,s."properties.transportMode" as transportMode
        ,s."properties.vehicles" as vehicles
        ,s."properties.direction" as direction
        ,s."geometry.type" as geometry_type
        ,s."geometry.coordinates" as geometry_coordinates
        from stg_conflicten.ndw_werkzaamheden_restriction_raw s
        WHERE s."properties.transportMode" = 'CAR'
            AND s."properties.restrictionType" = 'COMPLETE'
    ) as r
);'''
cur.execute(sql)

In [6]:
# multilinestring can not be read/transformed directly
# remove outer brackets using substring
# change squary to accolade brackets using translate
# use replace to indicate splitting points
# use regexp... to split multilinestrings to linestrings => generates extra rows
# read linestrings as GeoJSON
sql="""-- column with string of coordinates to geom
DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom;
CREATE TABLE int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom AS (
    SELECT melvin_id, restrictionType, transportMode, vehicles, direction, ST_SetSRID(geom,4326) as geom
    FROM (
        SELECT melvin_id, restrictionType, transportMode, vehicles, direction
        ,ST_GeomFromGeoJSON(json_build_object('type','LineString', 'coordinates', geom_multi::float[])) as geom
        FROM (
            SELECT melvin_id, restrictionType, transportMode, vehicles, direction
                ,regexp_split_to_table(REPLACE(translate(SUBSTRING("geometry_coordinates", 2, length("geometry_coordinates")-2), '[]', '{}'), '}}, {{', '}}bbb{{')::varchar,E'bbb') as geom_multi
            FROM int_conflicten.ndw_werkzaamheden_restriction_cleaned
            WHERE "geometry_type" = 'MultiLineString'
        ) as sub
        UNION
        -- linestring can be transformed into float array, and then made into a GeoJSON
        SELECT melvin_id, restrictionType, transportMode, vehicles, direction
            ,ST_GeomFromGeoJSON(json_build_object('type',"geometry_type", 'coordinates', translate("geometry_coordinates", '[]', '{}')::float[])) as geom
        FROM int_conflicten.ndw_werkzaamheden_restriction_cleaned
        WHERE "geometry_type" = 'LineString'
    ) as total
);"""
cur.execute(sql)


In [7]:
sql='''-- join wegvakID
-- inner join, as not the entire country is taken into consideration
DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom_wvid;
CREATE TABLE int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom_wvid AS (
    select r.*, wegvakken.id
    FROM int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom as r
    INNER JOIN prd_ndw_borden.nwb_wegvakken_bebording as wegvakken
    ON st_intersects(r.geom,wegvakken.geom)
);'''
cur.execute(sql)

## Periods table

In [8]:
sql='''DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_period_cleaned;
CREATE TABLE int_conflicten.ndw_werkzaamheden_period_cleaned AS (
    select melvin_id, timeperiod_startDate, timeperiod_endDate, period_id
    from (
        select distinct sit_id as melvin_id
            ,TO_TIMESTAMP(REPLACE(REPLACE(p."startDate", 'Z', ''), 'T', ' '),'YYYY-MM-DD HH24:MI:SS') AS timeperiod_startDate
            ,TO_TIMESTAMP(REPLACE(REPLACE(p."endDate", 'Z', ''), 'T', ' '),'YYYY-MM-DD HH24:MI:SS') AS timeperiod_endDate
            ,id as period_id
        from stg_conflicten.ndw_werkzaamheden_period_raw as p
    ) sub
    WHERE timeperiod_startDate::varchar NOT LIKE '0001%'
        AND timeperiod_endDate::varchar NOT LIKE '0001%'
);'''
cur.execute(sql)

## combine table
situation + road_id + periods

In [9]:
sql='''DROP TABLE IF EXISTS int_conflicten.ndw_werkzaamheden_wvid_cleaned;
CREATE TABLE int_conflicten.ndw_werkzaamheden_wvid_cleaned AS (
    select distinct s.melvin_id, r.id as road_id, s.city, s.roadname, s.roadauthority, s.roadauthoritytype, s.district
        , p.timeperiod_startDate, p.timeperiod_endDate
    from int_conflicten.ndw_werkzaamheden_situation_cleaned as s
    inner join int_conflicten.ndw_werkzaamheden_restriction_cleaned_geom_wvid as r
    on s.melvin_id=r.melvin_id
    inner join int_conflicten.ndw_werkzaamheden_period_cleaned as p
    on s.melvin_id=p.melvin_id
    ORDER BY s.melvin_id, p.timeperiod_startDate
);'''
cur.execute(sql)

In [10]:
sql='''DROP TABLE IF EXISTS int_conflicten.ndw_wegvakken_werkzaamheden;
CREATE TABLE int_conflicten.ndw_wegvakken_werkzaamheden AS (
    SELECT DISTINCT melvin.melvin_id::int as melvin_id, 
    id,source,target,spd,cost,1 AS c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom,
    melvin.timeperiod_startDate, melvin.timeperiod_endDate
    FROM prd_ndw_borden.nwb_wegvakken_bebording AS borden
    INNER JOIN int_conflicten.ndw_werkzaamheden_wvid_cleaned AS melvin
    ON borden.id=melvin.road_id
);'''
cur.execute(sql)